# False Positive Brand Classifier - Model Selection & Tuning

This notebook performs model selection and hyperparameter tuning for the FP classifier. It loads the feature transformer created in fp1_EDA_FE.ipynb and focuses on finding the optimal model.

## Objective
Select the best classification model and tune hyperparameters to maximize F2 score (recall-weighted).

**Important:** This notebook uses ONLY training and validation data. Test set evaluation, threshold optimization, and deployment are performed in fp3_model_evaluation_deployment.ipynb.

## Contents
1. [Setup](#setup)
2. [Data Loading & Split](#1-data-loading--split)
3. [Feature Transformation](#2-feature-transformation)
4. [Baseline Model Comparison](#3-baseline-model-comparison)
5. [Hyperparameter Tuning](#4-hyperparameter-tuning)
6. [Overfitting Analysis](#5-overfitting-analysis)
7. [Final Model Selection](#6-final-model-selection)

## Setup

In [1]:
# Standard imports
import json
import sys
import warnings
from pathlib import Path

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sp
import seaborn as sns

# Sklearn imports
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import LinearSVC

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Project imports
from src.fp1_nb.data_utils import load_jsonl_data, split_train_val_test
from src.fp1_nb.preprocessing import clean_text, create_text_features
from src.fp1_nb.feature_transformer import FPFeatureTransformer
from src.fp1_nb.modeling import (
    create_search_object,
    tune_with_logging,
    get_best_params_summary,
    get_best_model,
    compare_models,
    evaluate_model,
)
from src.fp2_nb.overfitting_analysis import (
    analyze_cv_train_val_gap,
    analyze_iteration_performance,
    get_gap_summary,
)

# Settings
warnings.filterwarnings('ignore')
# Suppress sklearn parallel warning (internal sklearn issue, not actionable)
warnings.filterwarnings('ignore', message='.*sklearn.utils.parallel.delayed.*')
pd.set_option('display.max_columns', 50)
plt.style.use('seaborn-v0_8-whitegrid')

# Configuration
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
TARGET_COL = 'is_sportswear'
N_FOLDS = 3

# Paths
DATA_PATH = project_root / 'data' / 'fp_training_data.jsonl'
MODELS_DIR = project_root / 'models'
IMAGES_DIR = project_root / 'images'

print("Setup complete!")

Setup complete!


## 1. Data Loading & Split

Load data and apply identical preprocessing and split as fp1 to ensure consistency.

In [ ]:
# Load the FP training data (same as fp1)
df = load_jsonl_data(DATA_PATH)

# Create combined text features (identical to fp1)
# This ensures we use the same preprocessing including metadata
df['text_features'] = create_text_features(
    df,
    text_col='content',
    title_col='title',
    brands_col='brands',
    source_name_col='source_name',
    category_col='category',
    include_metadata=True,
    clean_func=clean_text
)

print(f"\nText features created!")
print(f"Sample:\n{df['text_features'].iloc[0][:300]}...")

In [ ]:
# Split with stratification (identical to fp1 - same random_state ensures identical splits)
train_df, val_df, test_df = split_train_val_test(
    df,
    target_col=TARGET_COL,
    train_ratio=0.6,
    val_ratio=0.2,
    test_ratio=0.2,
    random_state=RANDOM_STATE
)

# Extract metadata for discrete metadata features (same as fp1)
train_source_names = train_df['source_name'].tolist()
train_categories = train_df['category'].tolist()
val_source_names = val_df['source_name'].tolist()
val_categories = val_df['category'].tolist()

## 2. Feature Transformation

Load the fitted feature transformer from fp1 and transform all splits.

In [4]:
# Load the fitted feature transformer from fp1
transformer_path = MODELS_DIR / 'fp_feature_transformer.joblib'
transformer = joblib.load(transformer_path)

# Load the transformer config
config_path = MODELS_DIR / 'fp_feature_config.json'
with open(config_path) as f:
    transformer_config = json.load(f)

print(f"Loaded transformer: {transformer}")
print(f"\nTransformer config:")
for key, value in transformer_config.items():
    print(f"  {key}: {value}")

Loaded transformer: FPFeatureTransformer(method='tfidf_word', max_features=10000, fitted=True)

Transformer config:
  method: tfidf_word
  max_features: 10000
  ngram_range: [1, 2]
  min_df: 2
  max_df: 0.95
  sublinear_tf: True
  norm: l2
  char_ngram_range: [3, 5]
  char_max_features: 5000
  lsa_n_components: 100
  context_window_words: 20
  doc2vec_vector_size: 100
  doc2vec_min_count: 2
  doc2vec_epochs: 40
  doc2vec_dm: 1
  doc2vec_window: 4
  sentence_model_name: all-MiniLM-L6-v2
  include_vocab_features: True
  vocab_window_size: 15
  proximity_window_size: 10
  random_state: 42


In [ ]:
# Transform train and validation splits using the fitted transformer
# NOTE: Test data is NOT used in this notebook - final evaluation is done in fp3
# Pass metadata for discrete metadata features (if transformer was fitted with metadata)
X_train = transformer.transform(
    train_df['text_features'],
    source_names=train_source_names,
    categories=train_categories
)
X_val = transformer.transform(
    val_df['text_features'],
    source_names=val_source_names,
    categories=val_categories
)

# Extract targets
y_train = train_df[TARGET_COL].values
y_val = val_df[TARGET_COL].values

# Combine train+val for hyperparameter tuning (CV will create internal train/val splits)
if sp.issparse(X_train):
    X_trainval = sp.vstack([X_train, X_val])
else:
    X_trainval = np.vstack([X_train, X_val])
y_trainval = np.concatenate([y_train, y_val])

# Combined metadata for hyperparameter tuning
trainval_source_names = train_source_names + val_source_names
trainval_categories = train_categories + val_categories

print(f"Feature dimensionality: {X_train.shape[1]}")
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_trainval shape: {X_trainval.shape} (for hyperparameter tuning)")
print(f"Metadata features enabled: {transformer.include_metadata_features}")
print(f"Metadata scaler fitted: {transformer._metadata_scaler is not None}")
print(f"\nNote: Test data reserved for final evaluation in fp3 notebook")

## 3. Baseline Model Comparison

Train and evaluate multiple classifiers on the transformed features to identify the best performing models for hyperparameter tuning.

In [6]:
# Cross-validation strategy
cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)
print(f"Using {N_FOLDS}-fold stratified CV")

# F2 scorer (weights recall 2x higher than precision)
f2_scorer = make_scorer(fbeta_score, beta=2)

Using 3-fold stratified CV


In [7]:
# Define baseline models
# Note: MultinomialNB not included as it requires non-negative features
# (sentence transformer embeddings contain negative values)
baseline_models = {
    'Logistic Regression': LogisticRegression(
        max_iter=1000, 
        random_state=RANDOM_STATE,
        class_weight='balanced'
    ),
    'Linear SVM': CalibratedClassifierCV(
        LinearSVC(max_iter=2000, random_state=RANDOM_STATE, class_weight='balanced'),
        cv=3
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=100,
        random_state=RANDOM_STATE,
        class_weight='balanced',
        n_jobs=-1
    ),
    'HistGradientBoosting': HistGradientBoostingClassifier(
        max_iter=100,
        max_depth=5,
        class_weight='balanced',
        random_state=RANDOM_STATE,
    ),
}

In [ ]:
# Train and evaluate baseline models
baseline_results = []

# Models that require dense arrays (not sparse matrices)
dense_required_models = {'HistGradientBoosting'}

for name, model in baseline_models.items():
    print(f"\nTraining {name}...")
    
    # Convert to dense if required
    if name in dense_required_models and sp.issparse(X_train):
        X_train_fit = X_train.toarray()
        X_val_fit = X_val.toarray()
    else:
        X_train_fit = X_train
        X_val_fit = X_val
    
    model.fit(X_train_fit, y_train)
    
    # Evaluate on validation set
    metrics = evaluate_model(
        model, X_val_fit, y_val,
        model_name=name,
        dataset_name='Validation',
        verbose=True,
        plot=False
    )
    
    # Add F2 score to metrics (recall-weighted)
    y_pred = model.predict(X_val_fit)
    metrics['f2'] = fbeta_score(y_val, y_pred, beta=2)
    print(f"  F2 Score:  {metrics['f2']:.4f} (recall-weighted)")
    
    baseline_results.append(metrics)

In [ ]:
# Compare baseline models (with F2 as primary metric)
baseline_comparison = compare_models(
    baseline_results,
    metrics_to_display=['f2', 'recall', 'precision', 'f1', 'accuracy', 'pr_auc'],
    title='Baseline Model Comparison (Validation Set)',
    save_path='images/fp_baseline_comparison.png'
)

## 4. Hyperparameter Tuning

Tune the top-performing baseline models using cross-validation on train+val combined (80% of data). The test set remains completely held out for final evaluation.

### 4.1 Logistic Regression Tuning

In [ ]:
# Logistic Regression parameter grid
# Simplified based on prior tuning: C=0.01-0.1 with class_weight=None works best
# L1/L2 penalty makes no difference at low C, so using l2 (simpler)
lr_param_grid = {
    'C': [0.01, 0.1],
    'penalty': ['l2'],
    'solver': ['lbfgs'],  # faster than saga for l2
    'class_weight': [None],
}

lr_search = create_search_object(
    search_type='grid',
    estimator=LogisticRegression(max_iter=2000, random_state=RANDOM_STATE),
    param_grid=lr_param_grid,
    cv=cv,
    refit='f2'
)

lr_search, lr_log, lr_csv = tune_with_logging(
    lr_search, X_trainval, y_trainval,
    model_name='logistic_regression'
)

### 4.2 Random Forest Tuning

In [ ]:
# Random Forest parameter grid
# Simplified based on prior tuning:
# - n_estimators=200 dominates top runs
# - max_depth=20/None perform similarly (removing 10)
# - min_samples_leaf=1 dominates (removing 2)
# - Both class_weight options work well
rf_param_grid = {
    'n_estimators': [200],
    'max_depth': [20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1],
    'class_weight': ['balanced', 'balanced_subsample'],
}

rf_search = create_search_object(
    search_type='grid',
    estimator=RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1),
    param_grid=rf_param_grid,
    cv=cv,
    refit='f2'
)

rf_search, rf_log, rf_csv = tune_with_logging(
    rf_search, X_trainval, y_trainval,
    model_name='random_forest'
)

### 4.3 HistGradientBoosting Tuning

In [ ]:
# HistGradientBoosting parameter grid
# Simplified based on prior tuning:
# - max_depth=5 clearly outperforms 3 and None
# - learning_rate=0.1 dominates top runs
# - min_samples_leaf=5 dominates (removing 20)
# - l2_regularization=0.0 is slightly better
# - Keeping max_iter variation to check convergence
hgb_param_grid = {
    'max_iter': [100, 200],
    'learning_rate': [0.1],
    'max_depth': [5],
    'min_samples_leaf': [5],
    'l2_regularization': [0.0],
    'class_weight': ['balanced'],
}

hgb_search = create_search_object(
    search_type='grid',
    estimator=HistGradientBoostingClassifier(random_state=RANDOM_STATE),
    param_grid=hgb_param_grid,
    cv=cv,
    refit='f2'
)

# HGB requires dense arrays - convert if sparse
X_trainval_hgb = X_trainval.toarray() if sp.issparse(X_trainval) else X_trainval

hgb_search, hgb_log, hgb_csv = tune_with_logging(
    hgb_search, X_trainval_hgb, y_trainval,
    model_name='hist_gradient_boosting'
)

### 4.4 Compare Tuned Models (CV Performance)

Compare models based on their cross-validation F2 scores. Model selection is based on CV performance, not test set performance.

In [ ]:
# Compare tuned models based on CV performance
tuned_models = {
    'LR_tuned': lr_search,
    'RF_tuned': rf_search,
    'HGB_tuned': hgb_search,
}

# Extract CV metrics for comparison
cv_comparison_data = []
for name, search in tuned_models.items():
    best_idx = search.best_index_
    cv_results = search.cv_results_
    
    metrics = {
        'model_name': name,
        'f2': cv_results['mean_test_f2'][best_idx],
        'recall': cv_results['mean_test_recall'][best_idx],
        'precision': cv_results['mean_test_precision'][best_idx],
        'f1': cv_results['mean_test_f1'][best_idx],
        'accuracy': cv_results['mean_test_accuracy'][best_idx],
        'pr_auc': cv_results['mean_test_average_precision'][best_idx],
    }
    cv_comparison_data.append(metrics)
    
    print(f"{name}: CV F2 = {metrics['f2']:.4f} (+/- {cv_results['std_test_f2'][best_idx]:.4f}), Recall = {metrics['recall']:.4f}, Precision = {metrics['precision']:.4f}")

In [ ]:
# Compare tuned models (F2 as primary metric)
tuned_comparison = compare_models(
    cv_comparison_data,
    metrics_to_display=['f2', 'recall', 'precision', 'f1', 'accuracy', 'pr_auc'],
    title='Tuned Model Comparison (CV Performance, Optimized for F2)',
    save_path='images/fp_tuned_comparison.png'
)

### 4.5 Top Hyperparameter Combinations

Review the top performing hyperparameter combinations for each model to guide future grid adjustments.

In [ ]:
def get_top_hyperparameter_runs(search_object, model_name, n_top=10, metric='f2'):
    """Extract top n hyperparameter combinations from a GridSearchCV object."""
    cv_results = search_object.cv_results_
    
    # Get parameter names (remove 'param_' prefix)
    param_names = [k.replace('param_', '') for k in cv_results.keys() if k.startswith('param_')]
    
    # Build dataframe with all results
    results_data = []
    for i in range(len(cv_results['mean_test_' + metric])):
        row = {
            'rank': cv_results['rank_test_' + metric][i],
            f'val_{metric}': cv_results['mean_test_' + metric][i],
            f'val_{metric}_std': cv_results['std_test_' + metric][i],
            f'train_{metric}': cv_results['mean_train_' + metric][i],
            'gap': cv_results['mean_train_' + metric][i] - cv_results['mean_test_' + metric][i],
        }
        # Add parameter values
        for param in param_names:
            row[param] = cv_results[f'param_{param}'][i]
        results_data.append(row)
    
    df = pd.DataFrame(results_data)
    df = df.sort_values('rank').head(n_top)
    
    return df, param_names

# Display top 10 hyperparameter combinations for each model
for model_name, search in tuned_models.items():
    print("=" * 80)
    print(f"TOP 10 HYPERPARAMETER COMBINATIONS: {model_name}")
    print("=" * 80)
    
    top_df, param_names = get_top_hyperparameter_runs(search, model_name, n_top=10, metric='f2')
    
    # Display with formatted output
    display_cols = ['rank', 'val_f2', 'val_f2_std', 'train_f2', 'gap'] + param_names
    print(top_df[display_cols].to_string(index=False))
    print()
    
    # Also show best parameters explicitly
    print(f"Best parameters: {search.best_params_}")
    print()

## 5. Overfitting Analysis

Analyze the gap between training and validation performance using CV results. This analysis uses only training data (through cross-validation) and does NOT use the held-out test set.

**Key insight**: In GridSearchCV, `mean_train_*` scores are computed on the training folds and `mean_test_*` scores are computed on the validation folds. A large gap indicates overfitting.

In [ ]:
# Get summary of train-validation gaps for all tuned models
gap_summary_df = get_gap_summary(tuned_models, metric='f2')

In [ ]:
# Detailed train-validation gap analysis for Random Forest
# Shows diagnosis (Good fit / Moderate / Severe overfitting) and recommendations
rf_gap_result = analyze_cv_train_val_gap(
    rf_search, 
    metric='f2',
    model_name='Random Forest',
    save_path='images/fp_rf_train_val_gap.png'
)

### 5.2 RF Performance vs n_estimators

Analyze how Random Forest performance changes with the number of trees, using optimal parameters from tuning.

In [ ]:
# Create a grid varying only n_estimators while keeping other RF params at optimal values
best_rf_params = rf_search.best_params_.copy()
n_estimators_values = [50, 100, 150, 200, 250, 300]

# Build param grid with all params fixed except n_estimators
rf_iteration_param_grid = {
    'n_estimators': n_estimators_values,
    'max_depth': [best_rf_params.get('max_depth')],
    'min_samples_split': [best_rf_params.get('min_samples_split', 2)],
    'min_samples_leaf': [best_rf_params.get('min_samples_leaf', 1)],
    'class_weight': [best_rf_params.get('class_weight', 'balanced')],
}

# Run grid search with this restricted grid
rf_iteration_search = create_search_object(
    search_type='grid',
    estimator=RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1),
    param_grid=rf_iteration_param_grid,
    cv=cv,
    refit='f2'
)

rf_iteration_search, _, _ = tune_with_logging(
    rf_iteration_search, X_trainval, y_trainval,
    model_name='rf_iterations'
)

In [ ]:
# Analyze performance across n_estimators values with confidence bands
# Shows train/val scores with gap annotation at optimal point
iteration_result = analyze_iteration_performance(
    rf_iteration_search,
    param_name='n_estimators',
    metric='f2',
    tuned_value=best_rf_params.get('n_estimators', 200),
    model_name='Random Forest',
    save_path='images/fp_rf_iteration_performance.png'
)

## 6. Final Model Selection

Select the best model based on CV F2 score and save artifacts for final evaluation in fp3.

**Note:** Test set evaluation, threshold optimization, and deployment are performed in fp3_model_evaluation_deployment.ipynb.

In [ ]:
# Select best model based on CV F2
best_model_name, best_model_metrics = get_best_model(tuned_comparison, 'f2')
print(f"Selected model: {best_model_name}")
print(f"CV F2: {best_model_metrics['f2']:.4f} (primary metric)")
print(f"CV Recall: {best_model_metrics['recall']:.4f}")
print(f"CV Precision: {best_model_metrics['precision']:.4f}")

# Get the best fitted model
best_search = tuned_models[best_model_name]
best_model = best_search.best_estimator_

# Save best classifier for fp3
classifier_path = MODELS_DIR / 'fp_best_classifier.joblib'
joblib.dump(best_model, classifier_path)
print(f"\nBest classifier saved to: {classifier_path}")

In [ ]:
# Save CV metrics for comparison in fp3
cv_metrics = {
    'model_name': best_model_name,
    'cv_f2': float(best_model_metrics['f2']),
    'cv_recall': float(best_model_metrics['recall']),
    'cv_precision': float(best_model_metrics['precision']),
    'cv_f1': float(best_model_metrics['f1']),
    'cv_accuracy': float(best_model_metrics['accuracy']),
    'cv_pr_auc': float(best_model_metrics['pr_auc']),
    'best_params': best_search.best_params_,
    'transformer_method': transformer.method,
    'n_folds': N_FOLDS,
}

cv_metrics_path = MODELS_DIR / 'fp_cv_metrics.json'
with open(cv_metrics_path, 'w') as f:
    json.dump(cv_metrics, f, indent=2)
print(f"CV metrics saved to: {cv_metrics_path}")

In [ ]:
print("=" * 60)
print("FP2 NOTEBOOK SUMMARY")
print("=" * 60)
print(f"\nBest Model: {best_model_name}")
print(f"Feature Method: {transformer.method}")
print(f"\nCV Performance ({N_FOLDS}-fold):")
print(f"  F2 Score:  {best_model_metrics['f2']:.4f}")
print(f"  Recall:    {best_model_metrics['recall']:.4f}")
print(f"  Precision: {best_model_metrics['precision']:.4f}")
print(f"\nBest Hyperparameters:")
for param, value in best_search.best_params_.items():
    print(f"  {param}: {value}")
print(f"\nSaved Artifacts for fp3:")
print(f"  - {classifier_path}")
print(f"  - {cv_metrics_path}")
print(f"\nNext: Run fp3_model_evaluation_deployment.ipynb for:")
print(f"  - Final test set evaluation")
print(f"  - Threshold optimization")
print(f"  - Pipeline export for deployment")
print("=" * 60)